# Support vector machines

In [1]:
# import libraries

import numpy as np
import math
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

## Data pre-processing

In [2]:
# load training data

# load data from csv file
df_train_features = pd.read_csv ('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

# Load test data
df_test_features = pd.read_csv ('test_features.csv')

 ### Histogram of the output labels 

We should check for class imbalance.

In [ ]:
labs.hist()

with PdfPages("./Results/Labels_histogram.pdf") as export_pdf:
    for i in list(labs)[1:]:
        labs.hist(column = i, bins = 100)
        export_pdf.savefig()

One can see the class imbalance problem here. Other observations:
  * Heartrate, RRate, ABPm,  distribution is similar to a normal distribution
  * SpO2 is like a censored normal distribution. 
  * For all of the other features, class imbalance is an obvious problem.

A basic strategy that could be used here: Upsample both classes! Do the upsampling efficiently, not just replicating the datapoints

### Train Data pre-processing

In [3]:
# data inspection: 
#############################################
# range of the provided data?
print(df_train_features.agg([min, max]))

# how much missing data? 
print("number of missing values:")
print(df_train_features.isnull().sum(axis=0))

       pid  Time    Age  EtCO2    PTT    BUN  Lactate  Temp   Hgb  HCO3  ...  \
min      1     1   15.0   10.0   12.5    1.0      0.2  21.0   3.3   0.0  ...   
max  31658   315  100.0  100.0  250.0  268.0     31.0  42.0  23.8  50.0  ...   

     Alkalinephos   SpO2  Bilirubin_direct  Chloride   Hct  Heartrate  \
min          12.0   20.0              0.01      66.0   9.4       23.0   
max        3833.0  100.0             21.20     141.0  63.4      191.0   

     Bilirubin_total  TroponinI   ABPs    pH  
min              0.1       0.01   21.0  6.82  
max             46.5     440.00  287.0  7.78  

[2 rows x 37 columns]
number of missing values:
pid                      0
Time                     0
Age                      0
EtCO2               218157
PTT                 217641
BUN                 207835
Lactate             217184
Temp                146825
Hgb                 205645
HCO3                215381
BaseExcess          208053
RRate                40155
Fibrinogen          22544

### Train data pre-processing

In [25]:
# aggregate data for each pid
# GROUPBY REARRANGES THE ROWS, WE HAVE TO DO THE SAME FOR THE LABELS
df_train_aggregate_features = df_train_features.groupby('pid').agg('sum')

# print(df_train_aggregate_features)

In [32]:
# remove time from data frame 
df_train_agg_features = df_train_aggregate_features.drop(['Time'], axis = 1)
# print(df_train_agg_features)

In [30]:
# impute missing data points
imp = SimpleImputer(strategy="mean")
df_train_agg_imputed_features = imp.fit_transform(df_train_agg_features)
#print(df_train_agg_imputed_features)

In [33]:
# REARRANGE THE LABELS, TO MATCH THE REARRANGED FEATURES
df_train_labels_sorted = df_train_labels.sort_values(by = 'pid')
# print(df_train_labels_sorted)

         pid  LABEL_BaseExcess  LABEL_Fibrinogen  LABEL_AST  \
0          1               1.0               0.0        0.0   
6622       2               0.0               0.0        0.0   
15008      4               0.0               0.0        0.0   
16335      6               1.0               0.0        0.0   
17676      8               0.0               0.0        0.0   
...      ...               ...               ...        ...   
14402  31653               0.0               0.0        0.0   
14403  31654               0.0               0.0        0.0   
14404  31656               1.0               0.0        0.0   
14405  31657               0.0               0.0        0.0   
14406  31658               0.0               0.0        0.0   

       LABEL_Alkalinephos  LABEL_Bilirubin_total  LABEL_Lactate  \
0                     0.0                    0.0            1.0   
6622                  0.0                    0.0            0.0   
15008                 0.0                 

### Test Data pre-processing

In [18]:
# data inspection: 
#############################################
# range of the provided data?
print(df_test_features.agg([min, max]))

# how much missing data? 
print("number of missing values:")
print(df_test_features.isnull().sum(axis=0))

       pid  Time    Age  EtCO2    PTT    BUN  Lactate  Temp   Hgb  HCO3  ...  \
min      0     1   16.0   10.0   18.1    1.0      0.3  27.0   2.3   0.0  ...   
max  31655   293  100.0  100.0  249.9  205.0     23.3  42.0  23.8  49.0  ...   

     Alkalinephos   SpO2  Bilirubin_direct  Chloride   Hct  Heartrate  \
min          11.0   22.0              0.01      74.0   9.1       21.0   
max        2121.0  100.0             21.00     145.0  71.7      184.0   

     Bilirubin_total  TroponinI   ABPs    pH  
min              0.1       0.01   28.0  6.62  
max             46.4     271.60  281.0  7.69  

[2 rows x 37 columns]
number of missing values:
pid                      0
Time                     0
Age                      0
EtCO2               145440
PTT                 145115
BUN                 138520
Lactate             144692
Temp                 97813
Hgb                 137096
HCO3                143546
BaseExcess          138730
RRate                26669
Fibrinogen          15031

### Test data pre-processing

In [20]:
# aggregate data for each pid
df_test_aggregate_features = df_test_features.groupby('pid').agg('sum')

#print(df_train_aggregate_features)

# collect all test pids
test_pids = list(set(df_test_features.pid))

In [21]:
# remove time from data frame 
df_test_agg_features = df_test_aggregate_features.drop(['Time'], axis = 1)
# print(df_test_agg_features)

In [22]:
# impute missing data points
df_test_agg_imputed_features = imp.transform(df_test_agg_features)

## Fit a model & Predict

### predict with support vector machine classification and use probabilities

In [35]:
# first for the labels that have an output [0,1]

columns = [test_pids]

for i in range(1, 2):
    clf = SVC(class_weight='balanced', max_iter = 10000)
    clf.fit(df_train_agg_imputed_features, df_train_labels_sorted.iloc[:,i])
    pred = clf.predict(df_test_agg_imputed_features)
    # proba = clf.predict_proba(df_test_agg_imputed_features)
    #columns.append(proba[:,1])
    

C:\Users\Naomi\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [ ]:
print(pred)

In [ ]:
print(columns)

In [ ]:
# labels that have a real value
for i in range(12, 16):
    clf = LinearSVR()
    clf.fit(df_train_agg_imputed_features, df_train_labels.iloc[:,i])
    pred = clf.predict(df_test_agg_imputed_features)
    columns.append(pred)

### predict with Support vector regression and then compute sigmoid function

In [ ]:
# first for the labels that have an output [0,1]

columns = [test_pids]

for i in range(1,12):
    clf = LinearSVR(fit_intercept = True, max_iter = 1000)
    clf.fit(df_train_agg_imputed_features, df_train_labels.iloc[:,i])
    pred = clf.predict(df_test_agg_imputed_features)
    prob = np.empty(len(pred))
    for i in range(0, len(pred)):
        prob[i] = 1 / (1 + math.exp(-pred[i]))
    columns.append(prob)

In [ ]:
# labels that have a real value
for i in range(12, 16):
    clf = LinearSVR()
    clf.fit(df_train_agg_imputed_features, df_train_labels.iloc[:,i])
    pred = clf.predict(df_test_agg_imputed_features)
    columns.append(pred)

## Save predictions

In [ ]:
print(np.shape(columns))
result = pd.DataFrame(columns).transpose()
result.columns = list(df_train_labels)
result.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')

In [ ]:
result.to_csv('prediction.csv', index=False, float_format='%.3f')